In [ ]:
!nvidia-smi

Thu Aug 11 02:05:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-g0wj1dg1
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-g0wj1dg1


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <cstdlib>
#include <time.h>
#include <iostream>
#include <string>
#include <math.h>
#include <assert.h>

#define BLOCK_SIZE 16

using namespace std;


__global__ void Convolution(float* A, float* B, float* C, int numARows, int numACols, int numBRows, int numBCols, int numCRows, int numCCols)
{
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;
    float sum;
   
    //A * B = C
    if(row<numCRows&&col<numCCols){
        sum =0;
        for(size_t i=0;i<numBCols ;i++){
            
            for(size_t j=0;j<numBRows;j++){
                sum+=A[row*numACols+i*numACols+col+j]*B[i*numBCols+j];
            }
    
        }
        C[row*numCCols+col]=sum;
    
    }    

}

__host__ void cpu_Convolution(float *A, float *B, float *C, int asize, int bsize){

	int csize=asize-bsize+1;
	int sum;
	for(int i=0;i<csize;i++){
		
		for (int j=0;j<csize;j++){
			sum=0;
			for(int k=0;k<bsize;k++){
				
				for(int l=0;l<bsize;l++){
					sum+=A[i*asize+k*asize+j+l]*B[k*bsize+l];

				}
				
			}
			C[i*csize+j]=sum;
		}



	}
	




}

void randomInit(float* data, int size)
{
	for (int i = 0; i < size; ++i)
		data[i] = rand() %10;
}

int main()
{
	srand(time(0));
	int a, b,c;
	cudaEvent_t start_G, stop_G;
	float gpu_miliseconds, cpu_miliseconds;
	cudaEventCreate(&start_G);
	cudaEventCreate(&stop_G);
	a=5; 	//size of the feature map 
	b=3;	//size of the filter
	c=a-b+1;	//size of the output matrix
	unsigned int size_A = a * a;
	unsigned int mem_size_A = sizeof(float) * size_A;
	float* h_A = (float*)malloc(mem_size_A);

	unsigned int size_B = b * b;
	unsigned int mem_size_B = sizeof(float) * size_B;
	float* h_B = (float*)malloc(mem_size_B);

	unsigned int size_C = c * c;
	unsigned int mem_size_C = sizeof(float) * size_C;
	float* h_C = (float*)malloc(mem_size_C);
	float* h_C_cpu = (float*)malloc(mem_size_C);
	randomInit(h_A, size_A);
	for (int i = 0; i < size_B; ++i)
    {
        h_B[i] = rand() %4;
    }

	float* d_A;
	float* d_B;
	float* d_C;
    //for (int i = 0; i < size_A; ++i)
    //{
    //    h_A[i] = i;
    //}
    
	cudaMalloc((void**)&d_A, mem_size_A);
	cudaMalloc((void**)&d_B, mem_size_B);
	cudaMalloc((void**)&d_C, mem_size_C);

	cudaMemcpy(d_A, h_A, mem_size_A, cudaMemcpyHostToDevice);
	cudaMemcpy(d_B, h_B, mem_size_B, cudaMemcpyHostToDevice);
	
	unsigned int grid_rows= (c+BLOCK_SIZE-1) / BLOCK_SIZE;
	unsigned int grid_cols= (c+BLOCK_SIZE-1) / BLOCK_SIZE;
	
	dim3 dimGrid(grid_cols,grid_rows);	
	dim3 dimBlock(BLOCK_SIZE,BLOCK_SIZE);
	
	cudaEventRecord(start_G,0);
	Convolution << < dimGrid, dimBlock >> >(d_A, d_B, d_C, a, a, b, b, c, c);



	
	cudaDeviceSynchronize();

	

	cudaEventRecord(stop_G,0);

	cudaEventSynchronize(stop_G);

	cudaMemcpy(h_C, d_C, mem_size_C, cudaMemcpyDeviceToHost);

	
	cudaEventElapsedTime(&gpu_miliseconds, start_G, stop_G);

	printf("\nTime took to compute matrix A(%d x %d) with filter B(%d x %x) on GPU is %f ms  \n \n", a, a,b,b, gpu_miliseconds);
    printf("matrix A\n");
	for (int i = 0;i < a;i++)
	{
		for (int j = 0;j < a;j++)
		{
			printf("%f\t", h_A[i*a + j]);
		}
		printf("\n");
	}printf("\n");
    printf("matrix B\n");
    for (int i = 0;i < b;i++)
	{
		for (int j = 0;j < b;j++)
		{
			printf("%f\t", h_B[i*b + j]);
		}
		printf("\n");
	}printf("\n");
    printf("matrix C\n");
	for (int i = 0;i < c;i++)
	{
		for (int j = 0;j < c;j++)
		{
			printf("%f\t", h_C[i*c + j]);
		}
		printf("\n");
	}
	cudaEventRecord(start_G, 0);
	cpu_Convolution(h_A, h_B, h_C_cpu, a,b);
	cudaEventRecord(stop_G,0);
	cudaEventSynchronize(stop_G);
	cudaEventElapsedTime(&cpu_miliseconds, start_G, stop_G);
	printf("\nTime took to compute matrix A(%d x %d) with filter B(%d x %x) on CPU is %f ms  \n \n", a, a,b,b, cpu_miliseconds);
    
	for (int i = 0;i < c;i++)
	{
		for (int j = 0;j < c;j++)
		{
			printf("%f\t", h_C_cpu[i*c + j]);
		}
		printf("\n");
	}

	free(h_A);
	free(h_B);
	free(h_C);
	free(h_C_cpu);
	cudaFree(d_A);
	cudaFree(d_B);
	cudaFree(d_C);

	return EXIT_SUCCESS;
}



Time took to compute matrix A(5 x 5) with filter B(3 x 3) on GPU is 0.028608 ms  
 
matrix A
4.000000	1.000000	2.000000	0.000000	0.000000	
7.000000	2.000000	7.000000	3.000000	3.000000	
3.000000	6.000000	3.000000	0.000000	0.000000	
6.000000	1.000000	4.000000	1.000000	6.000000	
3.000000	0.000000	9.000000	5.000000	3.000000	

matrix B
3.000000	0.000000	2.000000	
1.000000	0.000000	2.000000	
1.000000	1.000000	0.000000	

matrix C
46.000000	20.000000	22.000000	
51.000000	23.000000	35.000000	
32.000000	30.000000	39.000000	

Time took to compute matrix A(5 x 5) with filter B(3 x 3) on CPU is 0.002400 ms  
 
46.000000	20.000000	22.000000	
51.000000	23.000000	35.000000	
32.000000	30.000000	39.000000	

